In [1]:
import pygame
from pygame.locals import *
import numpy as np
class Renderer(object):
    
    def __init__(self, window_size=(1024, 768), render_fn=None, inplace=False, init=None, flags=None):
        self.window_size = window_size
        self.inplace = inplace
        self.render_fn = render_fn
        if init is not None:
            assert init.dtype == np.uint8 and init.shape == (window_size[0], window_size[1], 3), 'wrong type or shape for init'
            self.pixarray = init
        else:
            self.pixarray = np.zeros((window_size[0], window_size[1], 3), dtype=np.uint8)
        if flags is None:
            self.screen = pygame.display.set_mode(window_size)
        else:
            self.screen = pygame.display.set_mode(window_size, flags)

        
    def render_loop(self):
        ctr = 0
        try:
            while 1:
                for event in pygame.event.get():
                    if event.type == QUIT or (event.type == KEYUP and event.key in (K_ESCAPE, K_q)):
                       raise StopIteration()

                if self.render_fn is not None:
                    if self.inplace:
                        self.render_fn(buf=self.pixarray, frame_id=ctr)
                        new_pix = self.pixarray
                    else:
                        new_pix = self.render_fn(buf=self.pixarray, frame_id=ctr).astype(np.uint8)
                    pygame.surfarray.blit_array(self.screen, new_pix)
                    pygame.display.flip()
                    self.pixarray = new_pix
                    ctr += 1
        except StopIteration:
            pass
        finally:
            pygame.display.quit()

                
    def __call__(self, fn):
        "decorator sugar"
        self.render_fn = fn
        self.render_loop()
        return fn

In [2]:
import gaze_data
from Queue import Queue

class GazePoints(object):
    
    def __init__(self):
        self.q = Queue(maxsize=100)
        self.stream = gaze_data.DataStream(self.add_point)
        
    def add_point(self, x, y):
        self.q.put((x, y))
        
    def get(self):
        return self.q.get()

In [2]:
@Renderer(window_size=(1280, 1024))
def noise(buf, frame_id):
    return (np.random.rand(*buf.shape) * 255)

In [3]:
from PIL import Image
test_image = Image.open('/Users/bob/Downloads/15954928163_f05dd11e2e_k.jpg', 'r').convert('RGB')
test_image.thumbnail((1280, 1024))
test_image = np.asarray(test_image)
@Renderer(init=test_image, window_size=test_image.shape[:2])
def pulse(buf, frame_id):
    return (np.roll(buf, 9) | np.roll(buf, -9)) + 1


In [5]:
import threading
class Fixation(object):
    
    def __init__(self):
        self.proc = subprocess.Popen(['./MinimalFixationDataStream.exe'], stdout=subprocess.PIPE)
        self.reader_thread = threading.Thread(target=self.read_loop)
        self.reader_thread.start()
        self.engaged = False
        self.point = None
        
    def __del__(self):
        if getattr(self, 'proc'):
            self.proc.kill()

    def read_loop(self):
        for line in self.proc.stdout:
            split_line = line.split(':', 1)
            if len(split_line) != 2:
                continue
            k, data = split_line
            if k in ('Fixation Begin', 'Fixation Data'):
                vals = map(float, data.split('(')[1].split(')')[0].split(','))
                self.point = vals
                self.engaged = True
            elif k == 'Fixation End':
                self.engaged = False
                self.point = None
            if cnt > 10:
                break
        

In [6]:
fix = Fixation()

In [7]:
import time
for i in xrange(1000):
    print fix.point, fix.engaged
    time.sleep(1)

None False
None False
None False
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True
[916.8, 1207.5] True


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\bob\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\bob\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-5-b53015fecc5d>", line 28, in read_loop
    if cnt > 10:
NameError: global name 'cnt' is not defined



KeyboardInterrupt: 

In [55]:
del fix

In [43]:
cnt = 0
for point in gaze_data():
    if cnt > 100:
        break
    print point
    cnt += 1

Initialization was successful.

Press any key to exit...

The connection state is now TRYINGTOCONNECT (We are trying to connect to the EyeX Engine)

The connection state is now CONNECTED (We are connected to the EyeX Engine)

Waiting for gaze data to start streaming...

(892.0, 901.9, 2103902)
(894.1, 901.1, 16)
(895.8, 901.1, 14)
(896.8, 902.6, 16)
(901.2, 909.7, 15)
(908.5, 922.7, 14)
(914.7, 929.7, 32)
(913.9, 940.6, 12)
(915.6, 930.5, 17)
(921.7, 934.8, 15)
(921.7, 936.8, 14)
(929.3, 925.4, 17)
(917.1, 915.2, 15)
(903.4, 911.9, 13)
(898.5, 908.0, 17)
(897.0, 904.9, 29)
(897.6, 909.4, 15)
(889.8, 920.6, 15)
(891.0, 929.0, 13)
(895.2, 929.7, 23)
(899.7, 925.3, 17)
(901.0, 922.4, 12)
(902.1, 919.7, 15)
(905.6, 901.4, 11)
(905.7, 716.0, 31)
(908.1, 656.1, 14)
(921.0, 620.4, 15)
(925.9, 616.2, 15)
(938.8, 585.2, 15)
(937.9, 593.4, 14)
(942.2, 603.4, 16)
(940.3, 605.2, 15)
(936.5, 626.9, 20)
(933.2, 627.8, 34)
(935.4, 627.5, 12)
(932.4, 621.4, 12)
(929.1, 618.1, 15)
(927.6, 612.2, 12)
(9

In [3]:
points = GazePoints()
scratch = None
max_x = None
max_y = None
min_x = None
min_y = None
@Renderer(window_size=(0,0), flags=pygame.FULLSCREEN)
def gaze_point(**kwargs):
    global scratch, max_x, max_y, min_x, min_y
    display_info = pygame.display.Info()
    display_size = (display_info.current_w, display_info.current_h)
    if scratch is None:
        scratch = np.zeros((display_size[0], display_size[1], 3), dtype=np.uint8)
        scratch[:, :, :] = 255
    point = points.get()
    #x = min(point[0], display_size[0])
    #x = max(x, 10)
    #y = min(point[1], display_size[1])
    #y = max(y, 10)

    x = point[0]
    y = point[1]

    if max_x is None or x > max_x:
        max_x = x
    if max_y is None or y > max_y:
        max_y = y
    if min_x is None or x < min_x:
        min_x = x
    if min_y is None or y < min_y:
        min_y = y
    try:
        scratch[int(x), int(y), :] = (0, 0, 0)
    except:
        pass
   
    return scratch

loaded context 7450784


In [7]:
import gaze_data
from Queue import Queue

class PointQueue(object):
    
    def __init__(self):
        self.q = Queue(maxsize=100)
        self.stream = gaze_data.DataStream(self.add_point)
        
    def add_point(self, x, y):
        self.q.put((x, y))
        
    def get(self):
        return self.q.get()
        

In [2]:
points = PointQueue()

loaded context 8397216


In [6]:
import math
points = GazePoints()
@Renderer(window_size=(0,0), flags=pygame.FULLSCREEN)
def gaze_point(**kwargs):
    display_info = pygame.display.Info()
    display_size = (display_info.current_w, display_info.current_h)
    res = np.zeros((display_size[0], display_size[1], 3), dtype=np.uint8)
    point = points.get()
    frame_id = kwargs['frame_id']
    #point = (frame_id % display_size[0], frame_id % display_size[1])
    
    #euclidean distance field
    coords = np.mgrid[0:display_size[1], 0:display_size[0]].T
    dx = coords[:, :, 0] - point[1]
    dy = coords[:, :, 1] - point[0]
    distances = np.sqrt(dx**2 + dy**2)
    max_distance = np.max(distances)
    distance_ints = (distances / max_distance * 255).astype(np.uint8)
    
    res[:, :, 0] = distance_ints
    res[:, :, 1] = 128
    res[:, :, 2] = distance_ints
    
    return res

In [41]:
np.mgrid[0:100, 0:100].T[10, 10]

array([10, 10])

In [8]:
%load_ext Cython

In [1]:
import gaze_data

loaded context 8818704


In [3]:
points[-10:]

[(991.9627817264495, 1069.4232427384054),
 (976.9697991880523, 1073.0597348336373),
 (1066.5020475041263, 933.9038560232394),
 (1040.5993619601263, 997.5158789091838),
 (1002.9010723321159, 1050.443255032236),
 (985.3672512682385, 1048.39046722567),
 (943.5181883352036, 1088.432515618868),
 (933.9411604330621, 1129.5299685488092),
 (872.3828951285805, 1135.1213675726694),
 (859.3436666976396, 1115.8523639285088)]

In [10]:
del runner